# Practical Two

### [Neil D. Lawrence](http://inverseprobability.com), University of

Cambridge

### 2021-11-16

**Abstract**: In this practical session we look at the second aspect of
the Fynesse data science process, *assess*. In particular, we are going
to download data from OpenStreetMap and verify its quality and utility.

$$
$$

<!-- Do not edit this file locally. -->
<!-- Do not edit this file locally. -->
<!---->
<!-- Do not edit this file locally. -->
<!-- Do not edit this file locally. -->
<!-- The last names to be defined. Should be defined entirely in terms of macros from above-->
<!--

-->

## Revert Matplotlib Version

Revert `matplotlib` version on Google Colab due to bug mentioned
[here](https://github.com/facebook/prophet/issues/1691).

In [ ]:
%pip uninstall --yes matplotlib

In [ ]:
%pip install matplotlib==3.1.3

## Mapping Data

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_maps/includes/mapping-intro.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_maps/includes/mapping-intro.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

Many of the questions we ask in data science are about how we live, who
we are and how we might live better. So it is no surprise that an
important source of information for data science comes from *mapping
data*.

Questions about how best to make deliveries, where to locate facilities,
national disease susceptibility, how to get to work, where to go for a
walk, disaster response[1] and a myriad of other applications.

The wider field of geospatial data analysis is also vital for monitoring
our environment and human activity[2] on our globe.

With all of this in mind, mapping data seems an appropriate domain in
which to explore how we can assess a data set for use in a particular
data science domain.

In this practical session, you will learn how to download data from
OpenStreetMap, in particular we will focus on *points of interest* in
OpenStreetMap. We will explore the utility of points of interest as a
way of adding features into our models. We will introduce the notion of
a basis function. In spatial data the basis functions we use will become
features on the landscape.

[1] For example the [OpenStreetMap response to the 2010 Haiti
Earthquake](https://blog.openstreetmap.org/2010/01/14/haiti-openstreetmap-response/).

[2] For example this [UN Global
Pulse](https://www.unglobalpulse.org/project/measuring-poverty-with-machine-roof-counting/)
project which measures roof material as a proxy for poverty levels.

## Accessing Open Street Map

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_maps/includes/open-street-map.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_maps/includes/open-street-map.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

In [ ]:
%pip install osmnx

In [ ]:
import osmnx as ox
import matplotlib.pyplot as plt

We will center out download on the city of Lagos, which has the
following latitude and longitude.

In [ ]:
place_name = "Lagos, Nigeria"

latitude = 6.5244 # Lagos latitude
longitude = 3.3792 # Lagos longitude

In [ ]:
place_name = "Kampala, Uganda"

latitude = 0.347596 # Kampala latitude
longitude = 32.582520 # Kampala langitude

In [ ]:
placestub = place_name.lower().replace(' ', '-').replace(',','')

We’ll create a bounding box which is 0.2 degrees wide, 1 degree is
around 111km ([circumference of the Earth is around 40,000
km](https://en.wikipedia.org/wiki/Metre) and 40,000/360=111km).

In [ ]:
box_width = 0.2 # About 22 km
box_height = 0.2
north = latitude + box_height/2
south = latitude - box_width/2
west = longitude - box_width/2
east = longitude + box_width/2

Now we’ll download a set of points of interest from OpenStreetMap. We
can specify the points of interest we’re interested in by building a
small dictionary containing their labels as follows.

In [ ]:
# Retrieve POIs
tags = {"amenity": True, 
        "buildings": True, 
        "historic": True, 
        "leisure": True, 
        "shop": True, 
        "tourism": True}

Now we can use `osmx` to download all such points of interest within a
given bounding baox.

In [ ]:
pois = ox.geometries_from_bbox(north, south, east, west, tags)

That operation can take some time, particularly as the bounding box
grows larger. Once it is complete we can check how many points of
interest we have found.

::: {.cell .code}

``` python
```

:::{print(“There are {number} points of interest surrounding {placename}
latitude: {latitude}, longitude: {longitude}.”format(number=len(pois),
placename=place_name, latitude=latitude, longitude=longitude))

And then we can examine their contents in more detail.

In [ ]:
pois.info

We notice a few things.

1.  Points of interest do not have a consistent OpenStreetMap
    `element_type`, some are `node`, others are `relation` and we also
    have `way`. You can find out more about elements in OpenStreetMap on
    [this wiki page](https://wiki.openstreetmap.org/wiki/Elements). This
    will become important when tidying up the data for next stage
    processing.

2.  Many of the values are missing. In SQL we would express a missing
    value as `NULL`. But in `pandas` a missing value is expressed as
    not-a-number, `NaN`. This is quite a common standard, but it is not
    the only standard. Sometimes data is collected and coded with an
    “unreasonable” value for a missing value. For example, someone might
    set missing values for heights to -999. The concept is that this is
    an obviously void “height” and would trigger a human user to check
    whether it’s a missing value. Of course, this is obvious to humans,
    but not necessarily to a computer!

3.  

Nodes, ways and relations in OpenStreetMap all have different *keys*
associated with them. The data is not structured in standard database
columns. Different points of interest might have different keys present
or absent. The `gdf` we recover places these keys in its columns, but if

So we might be interested in the following keys.

In [ ]:
keys = ["name",
        "addr:city",
        "addr:postcode",
        "amenity",
        "building",
        "building:name",
        "building:colour",
        "building:material",
        "historic",
        "memorial",
        "religion",
        "tourism",
        "emergency",
        "leisure",
        "shop"]

But our downloaded `gdf` may have fewer keys.

In [ ]:
pois.columns.values

We can write a short piece of code to discover which keys are missing
drom the data frame’s columns.

In [ ]:
for key in keys:
    if key not in pois.columns:
        print(key)
        
present_keys = [key for key in keys if key in pois.columns]
pois[present_keys]

This gives us the relevant points of interest (part of the map). If we’d
like to see the entire street network, we can download the entire graph
from the location.

In [ ]:
graph = ox.graph_from_bbox(north, south, east, west)

# Retrieve nodes and edges
nodes, edges = ox.graph_to_gdfs(graph)

# Get place boundary related to the place name as a geodataframe
area = ox.geocode_to_gdf(place_name)

Which we can then render as follows.

<https://towardsdatascience.com/4-must-have-jupyterlab-extensions-for-geospatial-data-science-f3cf7822de4b>

In [ ]:
import matplotlib.pyplot as plt
import mlai
import mlai.plot as plot

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_figsize)

# Plot the footprint
area.plot(ax=ax, facecolor="white")

# Plot street edges
edges.plot(ax=ax, linewidth=1, edgecolor="dimgray")

# Plot buildings
#buildings.plot(ax=ax, facecolor="silver", alpha=0.7)

# Plot all POIs in Lagos
pois.plot(ax=ax, color="blue", alpha=0.7, markersize=10)
plt.tight_layout()
mlai.write_figure(directory="./maps", filename=placestub+"-pois.png")

<img class="" src="https://mlatcl.github.io/ads/./slides/diagrams//maps/kampala-uganda-pois.png" style="width:40%">

Figure: <i>Points of Interest as identified in Open Street Map.</i>

In [ ]:
tourist_places = pois[pois.tourism.notnull()]

In [ ]:
# Plot a subset of the POIs (e.g., tourist places)
# Create figure
fig, ax = plt.subplots(figsize=plot.big_figsize)

# Plot the footprint
area.plot(ax=ax, facecolor="white")

# Plot street edges
edges.plot(ax=ax, linewidth=1, edgecolor="dimgray")

# Plot tourist places 
tourist_places.plot(ax=ax, color="blue", alpha=1, markersize=50)
plt.tight_layout()
mlai.write_figure(directory="./maps", filename=placestub + "-tourist-sites.png")

<img class="" src="https://mlatcl.github.io/ads/./slides/diagrams//maps/lagos-tourist-sites.png" style="width:40%">

Figure: <i>Tourist sites identified as Points of Interest in Open Street
Map.</i>

We have the POIs that are associated with tourist places in a
geodataframe. To work with them in a machine learning algorithm, it will
be easier to conveert them to a `pandas` DataFrame. This means dealing
with the geometry. If we examine the POIs.

TK: Plot health centers from the NMIS data

In [ ]:
tourist_places.loc["node"]

In [ ]:
for i in tourist_places["geometry"]["way"]:
    print("latitude: {latitude}, longitude: {longitude}".format(latitude=i.centroid.y, longitude=i.centroid.x))

Next we convert the geodataframe of tourist places we’ve downloaded to a
DataFrame

# Basis Functions

## Basis Functions

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/basis-functions-intro.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/basis-functions-intro.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

Here’s the idea, instead of working directly on the original input
space, $\mathbf{ x}$, we build models in a new space,
$\boldsymbol{ \phi}(\mathbf{ x})$ where $\boldsymbol{ \phi}(\cdot)$ is a
*vector-valued* function that is defined on the space $\mathbf{ x}$.

## Quadratic Basis

Remember, that a *vector-valued function* is just a vector that contains
functions instead of values. Here’s an example for a one dimensional
input space, $x$, being projected to a *quadratic* basis. First we
consider each basis function in turn, we can think of the elements of
our vector as being indexed so that we have $$
\begin{align*}
\phi_1(x) & = 1, \\
\phi_2(x) & = x, \\
\phi_3(x) & = x^2.
\end{align*}
$$ Now we can consider them together by placing them in a vector, $$
\boldsymbol{ \phi}(x) = \begin{bmatrix} 1\\ x\\ x^2\end{bmatrix}.
$$ For the vector-valued function, we have simply collected the
different functions together in the same vector making them notationally
easier to deal with in our mathematics.

When we consider the vector-valued function for each data point, then we
place all the data into a matrix. The result is a matrix valued
function, $$
\boldsymbol{ \Phi}(\mathbf{ x}) = 
\begin{bmatrix} 1 & x_1 &
x_1^2 \\
1 & x_2 & x_2^2\\
\vdots & \vdots & \vdots \\
1 & x_n& x_n^2
\end{bmatrix}
$$ where we are still in the one dimensional input setting so
$\mathbf{ x}$ here represents a vector of our inputs with $n$ elements.

Let’s try constructing such a matrix for a set of inputs. First of all,
we create a function that returns the matrix valued function.

In [ ]:
import numpy as np

In [ ]:
def quadratic(x, **kwargs):
    """Take in a vector of input values and return the design matrix associated 
    with the basis functions."""
    return np.hstack([np.ones((x.shape[0], 1)), x, x**2])

## Functions Derived from Quadratic Basis

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/quadratic-basis.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/quadratic-basis.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

$$
f(x) = {\color{red}{w_0}} + {\color{magenta}{w_1 x}} + {\color{blue}{w_2 x^2}}
$$

In [ ]:
import matplotlib.pyplot as plt
import teaching_plots as plot

In [ ]:
f, ax = plt.subplots(figsize=plot.big_wide_figsize)
loc =[[0, 1.4,],
      [0, -0.7],
      [0.75, -0.2]]
text =['$\phi(x) = 1$',
       '$\phi(x) = x$',
       '$\phi(x) = x^2$']

plot.basis(quadratic, x_min=-1.3, x_max=1.3, 
           fig=f, ax=ax, loc=loc, text=text,
           diagrams='./ml')


<img src="https://mlatcl.github.io/ads/./slides/diagrams//ml/quadratic_basis002.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>The set of functions which are combined to form a *quadratic*
basis.</i>

In [ ]:
import pods
from ipywidgets import IntSlider

In [ ]:
pods.notebook.display_plots('quadratic_basis{num_basis:0>3}.svg', 
                            directory='./ml', 
                            num_basis=IntSlider(0,0,2,1))

This function takes in an $n\times 1$ dimensional vector and returns an
$n\times 3$ dimensional *design matrix* containing the basis functions.
We can plot those basis functions against there input as follows.

In [ ]:
# first let's generate some inputs
n = 100
x = np.zeros((n, 1))  # create a data set of zeros
x[:, 0] = np.linspace(-1, 1, n) # fill it with values between -1 and 1

Phi = quadratic(x)

fig, ax = plt.subplots(figsize=plot.big_wide_figsize)
ax.set_ylim([-1.2, 1.2]) # set y limits to ensure basis functions show.
ax.plot(x[:,0], Phi[:, 0], 'r-', label = '$\phi=1$', linewidth=3)
ax.plot(x[:,0], Phi[:, 1], 'g-', label = '$\phi=x$', linewidth=3)
ax.plot(x[:,0], Phi[:, 2], 'b-', label = '$\phi=x^2$', linewidth=3)
ax.legend(loc='lower right')
_ = ax.set_title('Quadratic Basis Functions')

The actual function we observe is then made up of a sum of these
functions. This is the reason for the name basis. The term *basis* means
‘the underlying support or foundation for an idea, argument, or
process,’ and in this context they form the underlying support for our
prediction function. Our prediction function can only be composed of a
weighted linear sum of our basis functions.

## Quadratic Functions

<img src="https://mlatcl.github.io/ads/./slides/diagrams//ml/quadratic_function002.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Functions constructed by weighted sum of the components of a
quadratic basis.</i>

In [ ]:
import pods
from ipywidgets import IntSlider

In [ ]:
pods.notebook.display_plots('quadratic_function{num_function:0>3}.svg', 
                            directory='./ml', 
                            num_function=IntSlider(0,0,2,1))

## Different Bases

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/basis-functions.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/basis-functions.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

Our choice of basis can be made based on what our beliefs about what is
appropriate for the data. For example, the polynomial basis extends the
quadratic basis to aribrary degree, so we might define the $j$th basis
function associated with the model as $$
\phi_j(x_i) = x_i^j
$$ which is known as the *polynomial basis*.

## Polynomial Basis

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/polynomial-basis.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/polynomial-basis.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

The polynomial basis combines higher order polynomials together to
create the function. For example the fourth order polynomial has five
components to its basis function. $$
\phi_j(x) = x^j
$$

In [ ]:
import numpy as np

In [ ]:
import mlai

In [ ]:
%load -n mlai.polynomial

In [ ]:
import matplotlib.pyplot as plt
import mlai
import teaching_plots as plot

In [ ]:
f, ax = plt.subplots(figsize=plot.big_wide_figsize)
f, ax = plt.subplots(figsize=plot.big_wide_figsize)
loc =[[0, 1.4,],
      [0, -0.7],
      [0.75, -0.2],
     [-0.75, -0.2],
     [-0.75, 2]]
text =['$\phi(x) = 1$',
       '$\phi(x) = x$',
       '$\phi(x) = x^2$',
       '$\phi(x) = x^3$',
       '$\phi(x) = x^4$']

plot.basis(mlai.polynomial, x_min=-1.3, x_max=1.3, 
           fig=f, ax=ax, loc=loc, text=text, num_basis=5,
           diagrams='./ml')

<img src="https://mlatcl.github.io/ads/./slides/diagrams//ml/polynomial_basis004.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>The set of functions which are combined to form a
*polynomial* basis.</i>

In [ ]:
import pods
from ipywidgets import IntSlider

In [ ]:
pods.notebook.display_plots('polynomial_basis{num_basis:0>3}.svg', 
                            directory='./ml', 
                            num_basis=IntSlider(0,0,4,1))

In [ ]:
pods.notebook.display_prediction(basis=mlai.polynomial, num_basis=5)

## Functions Derived from Polynomial Basis

$$
f(x) = {\color{red}{w_0}} + {\color{magenta}{w_1 x}} + {\color{blue}{w_2 x^2}} + {\color{green}{w_3 x^3}} + {\color{cyan}{w_4 x^4}}
$$

<img src="https://mlatcl.github.io/ads/./slides/diagrams//ml/polynomial_function002.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>A random combination of functions from the polynomial
basis.</i>

In [ ]:
import pods
from ipywidgets import IntSlider

In [ ]:
pods.notebook.display_plots('polynomial_function{func_num:0>3}.svg', 
                            directory='./ml', 
                            func_num=IntSlider(0,0,2,1))

To aid in understanding how a basis works, we’ve provided you with a
small interactive tool for exploring this polynomial basis. The tool can
be summoned with the following command.

In [ ]:
import pods

In [ ]:
pods.notebook.display_prediction(basis=mlai.polynomial, num_basis=5)

Try moving the sliders around to change the weight of each basis
function. Click the control box `display_basis` to show the underlying
basis functions (in red). The prediction function is shown in a thick
blue line. *Warning* the sliders aren’t presented quite in the correct
order. `w_0` is associated with the bias, `w_1` is the linear term,
`w_2` the quadratic and here (because we have four basis functions) we
have `w_3` for the *cubic* term. So the subscript of the weight
parameter is always associated with the corresponding polynomial’s
degree.

### Exercise 2

Try increasing the number of basis functions (thereby increasing the
*degree* of the resulting polynomial). Describe what you see as you
increase number of basis up to 10. Is it easy to change the function in
intiutive ways?

### Exercise 2 Answer

Write your answer to Exercise 2 here

In [ ]:
# Use this box for any code you need



## Different Basis

The polynomial basis is widely used in Engineering and graphics, but it
has some drawbacks in machine learning: outside the input region between
-1 and 1, the values of the polynomial basis rise very quickly.

Now we look at basis functions that have been used as the *activation*
functions in neural network model.

## Radial Basis Functions

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/radial-basis.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/radial-basis.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

Another type of basis is sometimes known as a ‘radial basis’ because the
effect basis functions are constructed on ‘centres’ and the effect of
each basis function decreases as the radial distance from each centre
increases.

$$
\phi_j(x) = \exp\left(-\frac{(x-\mu_j)^2}{\ell^2}\right)
$$

In [ ]:
import mlai

In [ ]:
%load -n mlai.radial

In [ ]:
import matplotlib.pyplot as plt
import mlai
import teaching_plots as plot

In [ ]:
f, ax = plt.subplots(figsize=plot.big_wide_figsize)

loc = [[-1.25, -0.4],
       [0., 1.25],
       [1.25, -0.4]]
text = ['$\phi_1(x) = e^{-(x + 1)^2}$',
        '$\phi_2(x) = e^{-2x^2}$', 
        '$\phi_3(x) = e^{-2(x-1)^2}$']
plot.basis(mlai.radial, x_min=-2, x_max=2, 
           fig=f, ax=ax, loc=loc, text=text,
           diagrams='./ml')

<img src="https://mlatcl.github.io/ads/./slides/diagrams//ml/radial_basis002.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>The set of functions which are combined to form the radial
basis.</i>

In [ ]:
import pods
from ipywidgets import IntSlider

In [ ]:
pods.notebook.display_plots('radial_basis{num_basis:0>3}.svg', 
                            directory='./ml', 
                            num_basis=IntSlider(0,0,2,1))

In [ ]:
pods.notebook.display_prediction(basis=mlai.radial, num_basis=3)

## Functions Derived from Radial Basis

$$
f(x) = \color{red}{w_1 e^{-2(x+1)^2}}  + \color{magenta}{w_2e^{-2x^2}} + \color{blue}{w_3 e^{-2(x-1)^2}}
$$

<img src="https://mlatcl.github.io/ads/./slides/diagrams//ml/radial_function002.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>A radial basis is made up of different locally effective
functions centered at different points.</i>

In [ ]:
from ipywidgets import IntSlider
import pods

In [ ]:
pods.notebook.display_plots('radial_function{func_num:0>3}.svg', 
                            directory='./ml', 
                            func_num=IntSlider(0,0,2,1))

## Rectified Linear Units

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/relu-basis.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/relu-basis.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

The rectified linear unit is a basis function that emerged out of the
deep learning community. Rectified linear units are popular in the
current generation of multilayer perceptron models, or deep networks.
These basis functions start flat, and then become linear functions at a
certain threshold. $$
\phi_j(x) = xH(v_j x+ v_0)
$$

In [ ]:
import numpy as np

In [ ]:
import mlai

In [ ]:
%load -n mlai.relu

In [ ]:
import matplotlib.pyplot as plt
import teaching_plots as plot
import mlai

In [ ]:
f, ax = plt.subplots(figsize=plot.big_wide_figsize)
loc =[[0, 1.4,],
      [-1, -0.5],
      [-0.33, 0.2],
      [0.33, -0.5],
      [1, 0.2]]
text =['$\phi(x) = 1$',
       '$\phi(x) = xH(x+1.0)$',
       '$\phi(x) = xH(x+0.33)$',
       '$\phi(x) = xH(x-0.33)$',
       '$\phi(x) = xH(x-1.0)$']
plot.basis(mlai.relu, x_min=-2.0, x_max=2.0, 
           fig=f, ax=ax, loc=loc, text=text,
           diagrams='./ml',
           num_basis=5)

<img src="https://mlatcl.github.io/ads/./slides/diagrams//ml/relu_basis004.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>The set of functions which are combined to form a rectified
linear unit basis.</i>

In [ ]:
import pods
from ipywidgets import IntSlider

In [ ]:
pods.notebook.display_plots('relu_basis{num_basis:0>3}.svg', 
                            directory='./ml', 
                            num_basis=IntSlider(0,0,4,1))

In [ ]:
pods.notebook.display_prediction(basis=mlai.relu, num_basis=5)

## Functions Derived from Relu Basis

$$
f(x) = \color{red}{w_0}   + \color{magenta}{w_1 xH(x+1.0) } + \color{blue}{w_2 xH(x+0.33) } + \color{green}{w_3 xH(x-0.33)} +  \color{cyan}{w_4 xH(x-1.0)}
$$

<img src="https://mlatcl.github.io/ads/./slides/diagrams//ml/relu_function002.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>A rectified linear unit basis is made up of different
rectified linear unit functions centered at different points.</i>

In [ ]:
import pods
from ipywidgets import IntSlider

In [ ]:
pods.notebook.display_plots('relu_function{func_num:0>3}.svg', 
                            directory='./ml', 
                            func_num=IntSlider(0,0,2,1))

## Hyperbolic Tangent Basis

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/hyperbolic-tangent-basis.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/hyperbolic-tangent-basis.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

The rectified linear unit is a basis function that used to be used a lot
for neural network models. It’s related to the sigmoid function by a
scaling. $$
\phi_j(x) = \tanh(v_j x+ v_0)
$$

In [ ]:
import numpy as np

In [ ]:
import mlai

In [ ]:
%load -n mlai.hyperbolic_tangent

Sigmoid or hyperbolic tangent basis was popular in the original
generation of multilayer perceptron models, or deep networks. These
basis functions start flat, rise and then saturate.

In [ ]:
import matplotlib.pyplot as plt
import teaching_plots as plot
import mlai

In [ ]:
f, ax = plt.subplots(figsize=plot.big_wide_figsize)
loc =[[0, 1.4,],
      [-1, -0.7],
      [-0.33, 0],
      [0.33, -0.7],
      [1, 0]]
text =['$\phi(x) = 1$',
       '$\phi(x) = \\tanh(x+1.0)$',
       '$\phi(x) = \\tanh(x+0.33)$',
       '$\phi(x) = \\tanh(x-0.33)$',
       '$\phi(x) = \\tanh(x-1.0)$']
plot.basis(mlai.hyperbolic_tangent, x_min=-2.0, x_max=2.0,
           fig=f, ax=ax, loc=loc, text=text,
           diagrams='./ml',
           num_basis=5)

<img src="https://mlatcl.github.io/ads/./slides/diagrams//ml/hyperbolic_tangent_basis004.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>The set of functions which are combined to form a *hyberbolic
tangent* basis.</i>

In [ ]:
import pods
from ipywidgets import IntSlider

In [ ]:
pods.notebook.display_plots('hyperbolic_tangent_basis{num_basis:0>3}.svg', 
                            directory='./ml', 
                            num_basis=IntSlider(0,0,4,1))

## Functions Derived from Tanh Basis

$$
f(x) = {\color{red}{w_0}}   + {\color{magenta}{w_1 \text{tanh}\left(x+1\right)}}  + {\color{blue}{w_2 \text{tanh}\left(x+0.33\right)}}  + {\color{green}{w_3 \text{tanh}\left(x-0.33\right)}} + {\color{cyan}{w_4 \text{tanh}\left(x-1\right)}}
$$

<img src="https://mlatcl.github.io/ads/./slides/diagrams//ml/hyperbolic_tangent_function002.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>A hyperbolic tangent basis is made up of s-shaped basis
functions centered at different points.</i>

In [ ]:
import pods
from ipywidgets import IntSlider

In [ ]:
pods.notebook.display_plots('hyperbolic_tangent_function{func_num:0>3}.svg', 
                            directory='./ml', 
                            func_num=IntSlider(0,0,2,1))

## Fourier Basis

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/fourier-basis.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_ml/includes/fourier-basis.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

[Joseph Fourier](https://en.wikipedia.org/wiki/Joseph_Fourier) suggested
that functions could be converted to a sum of sines and cosines. A
Fourier basis is a linear weighted sum of these functions. $$
f(x) = w_0  + w_1 \sin(x) + w_2 \cos(x) + w_3 \sin(2x) + w_4 \cos(2x)
$$

In [ ]:
import numpy as np

In [ ]:
import mlai

In [ ]:
%load -n mlai.fourier

In [ ]:
import matplotlib.pyplot as plt
import mlai
import teaching_plots as plot

In [ ]:
f, ax = plt.subplots(figsize=plot.big_wide_figsize)
loc =[[0., 0.4,],
      [0.5, 0.4],
      [1, -0.4],
      [1.25, 0.4],
      [1.5, -0.4]]
text =['$\phi(x) = 1$',
       '$\phi(x) = \sin(x)$',
       '$\phi(x) = \cos(x)$',
       '$\phi(x) = \sin(2x)$',
       '$\phi(x) = \cos(2x)$']
plot.basis(mlai.fourier, x_min=0, x_max=2, 
           fig=f, ax=ax, loc=loc, text=text,
           diagrams='./ml',
           num_basis=5)

<img src="https://mlatcl.github.io/ads/./slides/diagrams//ml/fourier_basis004.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>The set of functions which are combined to form a *Fourier*
basis.</i>

In [ ]:
import pods
from ipywidgets import IntSlider

In [ ]:
pods.notebook.display_plots('fourier_basis{num_basis:0>3}.svg', 
                            directory='./ml', 
                            num_basis=IntSlider(0,0,4,1))

In this code, basis functions with an *odd* index are sine and basis
functions with an *even* index are cosine. The first basis function
(index 0, so cosine) has a frequency of 0 and then frequencies increase
every time a sine and cosine are included.

In [ ]:
pods.notebook.display_prediction(basis=mlai.fourier, num_basis=5)

## Functions Derived from Fourier Basis

$$
f(x) = {\color{red}{w_0}}  + {\color{magenta}{w_1 \sin(x)}} + {\color{blue}{w_2 \cos(x)}} + {\color{green}{w_3 \sin(2x)}} + {\color{cyan}{w_4 \cos(2x)}}
$$

<img src="https://mlatcl.github.io/ads/./slides/diagrams//ml/fourier_function002.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>A random combination of functions from the Fourier basis.
Fourier basis is made up of sine and cosine functions with different
frequencies.</i>

In [ ]:
import pods
from ipywidgets import IntSlider

In [ ]:
pods.notebook.display_plots('fourier_function{func_num:0>3}.svg', 
                            directory='./ml', 
                            func_num=IntSlider(0,0,2,1))

## Lecture on Basis Functions from GPRS Uganda

In [ ]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('PoNbOnUnOao')

Figure: <i>Lecture on Basis functions from GPRS in Uganda in 2013.</i>

Then we create basis functions of different widths on the map, and look
at tourist hotspots.

## Thanks!

For more information on these subjects and more you might want to check
the following resources.

-   twitter: [@lawrennd](https://twitter.com/lawrennd)
-   podcast: [The Talking Machines](http://thetalkingmachines.com)
-   newspaper: [Guardian Profile
    Page](http://www.theguardian.com/profile/neil-lawrence)
-   blog:
    [http://inverseprobability.com](http://inverseprobability.com/blog.html)

## References